# Totally Random Trees Embedding

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [2]:
dataset = pd.read_csv('BostonHousing.csv')#spr co nie działa!!!
#X = dataset.iloc[:, 1:-1].values
#y = dataset.iloc[:, -1].values

In [3]:
y = dataset['medv']#jeden nawias kwadratowy zmienia df na vector
X = dataset.drop('medv',axis = 1)

In [4]:
#dataset.describe()

In [5]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
from sklearn.ensemble import AdaBoostRegressor
regressor = AdaBoostRegressor()
regressor.fit(X_train, y_train)

AdaBoostRegressor()

In [7]:
from sklearn.metrics import r2_score
r2_score(y_test, regressor.predict((X_test)))

0.7890462664786698

In [8]:
#grid defining
n_estimators = [int(x) for x in np.linspace(10, 1000, num = 2)] # number of trees in the random forest
#base_estimator = ['None'] # The base estimator from which the boosted ensemble is built
learning_rate = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 10, 50, 100, 1000, 10000] # Weight applied to each regressor at each boosting iteration
loss = ['linear', 'square', 'exponential'] # The loss function to use when updating the weights after each boosting iteration


random_grid = {'n_estimators': n_estimators,

#'base_estimator': base_estimator,

'learning_rate': learning_rate,

'loss': loss}

In [25]:
#Podobno pod WIn 8.1 n_jobs inne niż 1 nie dziła z verbose, wszystkie rdzenie i tak sa zaangażowane
from sklearn.model_selection import RandomizedSearchCV
regressor_hpt_stochastic = RandomizedSearchCV(estimator = regressor,param_distributions = random_grid,n_iter = 10, cv = 10, verbose=3, random_state=35, n_jobs = 1, error_score='raise')
regressor_hpt_stochastic.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=10, error_score='raise', estimator=AdaBoostRegressor(),
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.1, 0.2, 0.3, 0.4,
                                                          0.5, 0.6, 0.7, 0.8,
                                                          0.9, 1, 10, 50, 100,
                                                          1000, 10000],
                                        'loss': ['linear', 'square',
                                                 'exponential'],
                                        'n_estimators': [10, 1000]},
                   random_state=35, verbose=3)

# Print the best parameters

In [26]:
print ('Random grid: ', random_grid, '\n')
# print the best parameters
print ('Best Parameters: ', regressor_hpt_stochastic.best_params_, ' \n')

Random grid:  {'n_estimators': [10, 1000], 'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 10, 50, 100, 1000, 10000], 'loss': ['linear', 'square', 'exponential']} 

Best Parameters:  {'n_estimators': 10, 'loss': 'exponential', 'learning_rate': 1}  



# Using the best parameters

In [27]:
best_regressor = AdaBoostRegressor(n_estimators= 1000, loss= 'square', learning_rate= 0.9) 
best_regressor.fit( X_train, y_train) 

AdaBoostRegressor(learning_rate=0.9, loss='square', n_estimators=1000)

In [28]:
y_pred_rf1 = pd.DataFrame({ "actual": y_test, "predicted_prob": best_regressor.predict((X_test))}) 
y_pred_rf1

,actual,predicted_prob
173,23.6,22.009804
274,32.4,32.000000
491,13.6,16.117021
72,22.8,24.753846
452,16.1,18.589785
...,...,...
110,21.7,20.855556
321,23.1,24.452174
265,22.8,24.610241
29,21.0,21.284756


In [29]:
#from sklearn.metrics import r2_score
r2_score(y_test, best_regressor.predict((X_test)))

0.8106223744637384